In [1]:
import json as js # for reading json files
import pandas as pd # for dataframe structures usage
import numpy as np # for mathematical calculations over dataframes

### Here I'm reading the json file

In [2]:
a = open('SPY_15min_2018.json')
a = a.read()
a = js.loads(a)

### Here I'm processing the json file to extract the information to build a dataframe

In [3]:
# Extract the info from json file
SPX_open_bid = [a[x]["openPrice"]["bid"] for x in range(len(a))]
SPX_open_ask = [a[x]["openPrice"]["ask"] for x in range(len(a))]
SPX_high_bid = [a[x]["highPrice"]["bid"] for x in range(len(a))]
SPX_high_ask = [a[x]["highPrice"]["ask"] for x in range(len(a))]
SPX_low_bid = [a[x]["lowPrice"]["bid"] for x in range(len(a))]
SPX_low_ask = [a[x]["lowPrice"]["ask"] for x in range(len(a))]
SPX_close_bid = [a[x]["closePrice"]["bid"] for x in range(len(a))]
SPX_close_ask = [a[x]["closePrice"]["ask"] for x in range(len(a))]
SPX_time = [a[x]["snapshotTime"] for x in range(len(a))]

### Here I pick the BID prices for the calculations (It is the same with the ASK prices)

In [4]:
# Build the SPX_BID dataframe
SPX_BID = pd.DataFrame(np.transpose(np.matrix([SPX_open_bid,SPX_high_bid,SPX_low_bid,SPX_close_bid])))
SPX_BID.columns = ['Open', 'High', 'Low', 'Close']
SPX_BID.index = SPX_time
SPX_BID.index = pd.to_datetime(SPX_BID.index)

### For the example, I chose the first 10 candels as the "last" known candels

In [5]:
Hist_Data = SPX_BID.iloc[0:10,:]
Hist_Data

,Open,High,Low,Close
2018-12-02 23:00:00,2795.47,2806.6,2794.85,2806.1
2018-12-02 23:15:00,2806.35,2808.6,2804.35,2804.6
2018-12-02 23:30:00,2804.35,2804.6,2801.6,2802.22
2018-12-02 23:45:00,2802.1,2803.35,2801.1,2802.1
2018-12-03 00:00:00,2802.35,2803.1,2799.35,2799.6
2018-12-03 00:15:00,2799.85,2800.85,2798.85,2798.85
2018-12-03 00:30:00,2799.1,2800.6,2798.35,2798.35
2018-12-03 00:45:00,2798.47,2799.47,2797.35,2798.1
2018-12-03 01:00:00,2797.85,2802.1,2797.85,2801.6
2018-12-03 01:15:00,2801.47,2802.1,2800.1,2800.6


### Here I define the function

In [6]:
def get_AIBD_signal(Hist_Data, power = 0.75, Threshold_ranges = [-0.2, 0.5]):
    time_window = Hist_Data.shape[0] # It identifies the time window for past data you're using
    n = time_window**power # this is one of the parameters of the algoritm
    
    High_shift = Hist_Data['High'].shift(1) # For the current candel, I consider the last 9 candels for calculations in the High
    Low_shift = Hist_Data['Low'].shift(1) # Same for the Low
    TR1 = High_shift-Low_shift # Calculate the first value of reference as an array
    TR2 = np.abs(High_shift - Hist_Data['Close']) # Calculate the second value of reference as an array
    TR3 = np.abs(Low_shift-Hist_Data['Close']) # Calculate the third value of reference as an array
    TR = pd.concat([TR1,TR2,TR3], axis = 1) # join them into a dataframe
    TR = [max(TR.iloc[i,:]) for i in range(TR.shape[0])] # For each row, take the maximum value
    Avr_TR = np.nanmean(TR) # calculate the mean of TR
    High_Range = Hist_Data['High'] - Hist_Data['Low'].min() # Get the difference between the current High and the past min Low
    Low_Range = Hist_Data['Low'] - Hist_Data['High'].max() # Get the difference between the current Low and the past max High
    NHR = High_Range/n/Avr_TR
    NHR = NHR.values[-1] # Extracts the current value of the indicator for the High
    NLR = Low_Range/n/Avr_TR
    NLR = NLR.values[-1] # Extracts the current value of the indicator for the Low
    
    # Defines the value of the signal according to the trading rule
    if (NHR > Threshold_ranges[1] and NLR < Threshold_ranges[0]):
        signal = 1
    elif (NLR > Threshold_ranges[0] and NHR < Threshold_ranges[1]):
        signal = -1
    else:
        signal = 0

    return(signal)
    

In [8]:
get_AIBD_signal(Hist_Data)

0